In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.sparse import csr_matrix
import sklearn
import itertools
import scipy as sp
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv("movie_recommendation.csv")
pd.set_option('display.max_columns', None)
movies = pd.read_csv("data/movies2.csv")
tags = pd.read_csv("data/tags2.csv")
ratings = pd.read_csv("data/ratings2.csv")
links = pd.read_csv("data/links2.csv")

In [3]:
data

,userId,movieId,rating,date,title,genres,link,published year
0,1,1,4.0,30-07-00,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,www.imdb.com/title/tt114709,(1995)
1,5,1,4.0,08-11-96,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,www.imdb.com/title/tt114709,(1995)
2,7,1,4.5,25-01-05,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,www.imdb.com/title/tt114709,(1995)
3,15,1,2.5,13-11-17,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,www.imdb.com/title/tt114709,(1995)
4,17,1,4.5,18-05-11,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,www.imdb.com/title/tt114709,(1995)
...,...,...,...,...,...,...,...,...
100831,610,160341,2.5,19-11-16,Bloodmoon (1997),Action|Thriller,www.imdb.com/title/tt118745,(1997)
100832,610,160527,4.5,19-11-16,Sympathy for the Underdog (1971),Action|Crime|Drama,www.imdb.com/title/tt66806,(1971)
100833,610,160836,3.0,03-05-17,Hazard (2005),Action|Drama|Thriller,www.imdb.com/title/tt798722,(2005)
100834,610,163937,3.5,04-05-17,Blair Witch (2016),Horror|Thriller,www.imdb.com/title/tt1540011,(2016)


## Naiev collaborative filltering

In [4]:
movie_user_mat = data.pivot(index='movieId', columns='userId', values='rating').fillna(0)

In [5]:
movie_user_mat

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,4.5,3.5,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,3.0,4.0,5.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,4.0,0.0,2.5,0.0,0.0,5.0,0.0,4.5,0.0,0.0,0.5,0.0,4.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,4.0,0.0,0.0,3.0,3.0,4.0,0.0,3.0,0.0,0.0,5.0,0.0,4.5,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,3.0,4.0,0.0,4.0,0.0,0.0,3.0,4.0,0.0,0.0,3.5,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,2.0,0.0,3.0,4.0,0.0,0.0,4.5,4.0,4.0,0.0,0.0,0.0,0.0,5.0,3.5,0.0,4.5,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,5.0,4.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,3.0,0.0,3.0,4.0,0.0,3.5,5.0,0.0,0.0,3.5,0.0,0.0,3.5,0.0,0.0,5.0,0.0,0.0,3.5,3.0,5.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,4.5,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0

In [6]:
movie_user_mat_sparse = csr_matrix(movie_user_mat.values)

In [7]:
movie_user_mat_sparse

<9724x610 sparse matrix of type '<class 'numpy.float64'>'
	with 100836 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.neighbors import NearestNeighbors
# define model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
# fit
model_knn.fit(movie_user_mat_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [9]:
df_movies = movies[["movieId","title"]]

In [10]:
movie_to_idx = {
    movie: i for i, movie in 
    enumerate(list(df_movies.set_index('movieId').loc[movie_user_mat.index].title))
}



In [11]:
from fuzzywuzzy import fuzz

def fuzzy_matching(mapper, fav_movie, verbose=True):
    """
    return the closest match via fuzzy ratio. 
    
    Parameters
    ----------    
    mapper: dict, map movie title name to index of the movie in data
    fav_movie: str, name of user input movie
    
    verbose: bool, print log if True
    Return
    ------
    index of the closest match
    """
    match_tuple = []
    # get match
    for title, idx in mapper.items():
        ratio = fuzz.ratio(title.lower(), fav_movie.lower())
        if ratio >= 60:
            match_tuple.append((title, idx, ratio))
    # sort
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('Oops! No match is found')
        return
    if verbose:
        print('Found possible matches in our database: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]

In [12]:
def make_recommendation(model_knn, data, mapper, fav_movie, n_recommendations):
    """
    return top n similar movie recommendations based on user's input movie
    Parameters
    ----------
    model_knn: sklearn model, knn model
    data: movie-user matrix
    mapper: dict, map movie title name to index of the movie in data
    fav_movie: str, name of user input movie
    n_recommendations: int, top n recommendations
    Return
    ------
    list of top n similar movie recommendations
    """
    # fit
    model_knn.fit(data)
    
    # get input movie index
    idx = fuzzy_matching(mapper, fav_movie, verbose=True)
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    
    raw_recommends = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    # print recommendations
    print('Recommendations for {}:'.format(fav_movie))
    test_array = []
    for i, (idx, dist) in enumerate(raw_recommends):
        test_array.append(reverse_mapper[idx])
    
    return test_array

In [13]:
test_array = []

def test(my_favorite):
        test_array.append(make_recommendation(
        model_knn=model_knn,
        data=movie_user_mat_sparse,
        fav_movie=my_favorite,
        mapper=movie_to_idx,
        n_recommendations=10))
    
my_favorite = ["toy story"]
for movie in my_favorite:
    test(movie)

Found possible matches in our database: ['Toy Story (1995)', 'Toy Story 3 (2010)', 'Toy Story 2 (1999)']

Recommendations for toy story:


In [14]:
test_array

[['Back to the Future (1985)',
  'Groundhog Day (1993)',
  'Mission: Impossible (1996)',
  'Star Wars: Episode VI - Return of the Jedi (1983)',
  'Lion King, The (1994)',
  'Forrest Gump (1994)',
  'Star Wars: Episode IV - A New Hope (1977)',
  'Independence Day (a.k.a. ID4) (1996)',
  'Jurassic Park (1993)',
  'Toy Story 2 (1999)']]

## -----------------------------------------------------------------------------------------------

## ----------------------------------------------------------------------------------------------

In [15]:
data2 = data[["userId", "movieId", "rating"]]
group1 = data2.groupby(['userId'],as_index = False, sort=False)["rating"].sum()
data2 = pd.merge(data2, group1, on = "userId", how = "left", sort = False)
group2 = data2.groupby(['userId'],as_index = False, sort=False)["rating_x"].count()
data2 = pd.merge(data2, group2, on = "userId", how = "left", sort = False)
data2["normalize_rating"] = (data2["rating_x_x"] - data2["rating_y"]) / data2["rating_x_y"]
data_normalize = data2.rename(columns = {"rating_x_x":"rating", "rating_y":"sum", "rating_x_y":"count"})

In [16]:
user_similarity_mat = data_normalize.pivot(index='movieId', columns='userId', values='normalize_rating').fillna(0)
user_similarity_mat.reset_index(inplace=True)
user_similarity_mat = user_similarity_mat.drop('movieId', axis=1)
user_ratings = pd.DataFrame(user_similarity_mat, index = user_similarity_mat.columns, columns = user_similarity_mat.columns)
similarity = cosine_similarity(user_ratings)
user_ratings = pd.DataFrame(similarity, index = user_ratings.index, columns = user_ratings.index)

#piv_sparse = sp.sparse.csr_matrix(piv_norm.values)
#user_similarity = 1 - spatial.distance.cosine(piv_sparse)
#user_similarity = cosine_similarity(piv_sparse)


for i in range(len(user_ratings.columns)) :
    for j in range(len(user_ratings.columns)) :
        user_ratings.iloc[i,j] = 1-cosine(user_similarity_mat.iloc[:,i],user_similarity_mat.iloc[:,j])

In [17]:
user_ratings

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.307150,0.094464,0.293686,0.314981,0.218111,0.183584,0.050789,0.431272,0.316568,0.165862,0.200028,0.082739,0.229930,0.273202,0.227931,0.193172,0.496443,0.148479,0.300395,0.244369,0.129126,0.195439,0.192729,0.171878,0.178045,0.111350,0.174855,0.071289,0.236082,0.391000,0.451195,0.221790,0.167966,0.381170,0.000000,0.134511,0.055355,0.148627,0.291744,0.133917,0.215408,0.412460,0.420989,0.104014,0.325481,0.089549,0.000000,0.150543,0.000000,0.116616,0.145360,0.338880,0.097083,0.298737,0.107903,0.023709,0.213149,0.118077,0.000000,0.127379,0.221904,0.041304,0.169464,0.220867,0.029857,0.066633,0.183403,0.000000,0.049906,0.050539,0.00000,0.124425,0.030519,0.0,0.022050,0.176157,0.221921,0.193811,0.132796,0.186716,0.234372,0.045784,0.311766,0.092780,0.051124,0.000000,0.090769,0.200987,0.064149,0.044069,0.391010,0.252788,0.0,0.260287,0.000000,0.412071,0.241100,0.200576,0.052608,0.038630,0.093666,0.116737,0.065050,0.000000,0.171566,0.050372,0.075121,0.125036,0.0,0.0,0.000000,0.196629,0.0,0.119644,0.297633,0.121511,0.344949,0.1073

def sim_movies(users, user_movie):
    movies = []
    for user in users:
        movies.append(data[data["userId"] == user[0]]["title"])
    movies = list(itertools.chain.from_iterable(movies))
    movies = list(set(movies))
    movies = set(movies) - set(user_movie)
    return movies

In [18]:
def top_users(user):
    users = []
    #if user not in user_sim_df.columns:
      #  return('No data available on user {}'.format(user))
    
    sim_values = user_ratings.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:11]
    sim_users = user_ratings.sort_values(by=user, ascending=False).index[1:11]
    zipped = zip(sim_users, sim_values,)
    for user, sim in zipped:
        users.append(user)
    #movies = sim_movies(users, user_movie)
    return users

In [19]:
sim_user = top_users(1)
#sim_user.append(1.0)
sim_user_data = pd.DataFrame()
sim_user.append(1)

for user in sim_user:
    sim_user_data = sim_user_data.append(data[data["userId"] == user])

### -----------------------------------------------------------------------------------------------------------------------
### -----------------------------------------------------------------------------------------------------------------------
## movie based

In [20]:
sim_user_data = sim_user_data.drop_duplicates()

In [21]:
movie_similarity_mat = sim_user_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)
movie_similarity_mat

movieId  1       2       3       5       6       7       8       10      \
userId                                                                    
1           4.0     0.0     4.0     0.0     4.0     0.0     0.0     0.0   
18          3.5     3.0     0.0     0.0     4.0     0.0     0.0     0.0   
276         4.0     4.0     0.0     3.0     0.0     4.0     0.0     0.0   
322         3.5     3.0     0.0     0.0     0.0     0.0     0.0     4.0   
325         0.0     0.0     0.0     0.0     4.0     0.0     0.0     0.0   
418         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
436         4.0     4.0     0.0     0.0     0.0     0.0     3.0     0.0   
483         4.0     4.0     0.0     2.5     0.0     0.0     0.0     2.0   
504         4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
506         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
512         0.0     3.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  11      12      16      18      19      22      23      24      \
userId                                                                    
1           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
18          0.0     0.0     4.5     0.0     0.0     0.0     0.0     0.0   
276         0.0     4.0     0.0     0.0     4.0     0.0     0.0     0.0   
322         4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
325         0.0     0.0     0.0     0.0     2.0     3.0     0.0     0.0   
418         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
436         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
483         0.0     0.0     4.0     4.0     3.5     0.0     3.0     2.5   
504         0.0     0.0     0.0     0.0     0.0     3.5     0.0     0.0   
506         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
512         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  26      27      29      30      31      32      34      36      \
userId                                                                    
1           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
18          0.0     0.0     0.0     0.0     0.0     4.0     2.5     4.0   
276         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
322         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
325         0.0     0.0     4.0     4.0     0.0     4.0     0.0     0.0   
418         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
436         0.0     0.0     0.0     0.0     0.0     0.0     5.0     0.0   
483         4.0     0.0     5.0     0.0     2.0     5.0     0.0     0.0   
504         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
506         0.0     3.0     0.0     0.0     0.0     0.0     0.0     0.0   
512         0.0     0.0     0.0     0.0     0.0     5.0     0.0     0.0   

movieId  39      47      48      50      58      60      62      70      \
userId                                                                    
1           0.0     5.0     0.0     5.0     0.0     0.0     0.0     3.0   
18          0.0     4.5     0.0     5.0     0.0     0.0     0.0     3.5   
276         0.0     0.0     0.0     0.0     0.0     0.0     5.0     0.0   
322         0.0     4.0     0.0     4.0     0.0     0.0     0.0     0.0   
325         0.0     0.0     0.0     0.0     4.0     0.0     0.0     0.0   
418         0.0     0.0     0.0     4.5     0.0     0.0     0.0     0.0   
436         0.0     0.0     4.0     0.0     0.0     3.0     0.0     0.0   
483         0.0     3.5     3.0     4.5     0.0     0.0     0.0     0.0   
504         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
506         5.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
512         4.0     5.0     0.0     5.0     0.0     0.0     0.0     0.0   

movieId  80      81      88      101     104     110     111     135     \
userId               

In [22]:
df = movie_similarity_mat.replace(0, np.NaN)
df = df.mean()

In [23]:
for column in movie_similarity_mat.columns:
    movie_similarity_mat[column][1] = df[column]

In [24]:
movie_similarity_mat.reset_index(inplace=True)
movie_similarity_mat = movie_similarity_mat.drop('userId', axis=1)

In [25]:
movie_ratings = pd.DataFrame(movie_similarity_mat, index = movie_similarity_mat.columns, columns = movie_similarity_mat.columns)

In [26]:
movie_ratings2 = movie_ratings.fillna(0)

In [27]:
similarity = cosine_similarity(movie_ratings2)

for i in range(len(movie_ratings.columns)) :
    for j in range(len(movie_ratings.columns)) :
        movie_ratings.iloc[i,j] = 1-cosine(movie_similarity_mat.iloc[:,i],movie_similarity_mat.iloc[:,j])

In [28]:
movie_ratings2 = pd.DataFrame(similarity, index = movie_ratings2.index, columns = movie_ratings2.index)

In [29]:
movie_ratings2

movieId    1         2         3         5         6         7         8       \
movieId                                                                         
1        1.000000  0.133822  0.263842  0.290436  0.123769  0.376838  0.074120   
2        0.133822  1.000000  0.092043  0.041641  0.157762  0.155136  0.015585   
3        0.263842  0.092043  1.000000  0.224758  0.106698  0.250281  0.125193   
5        0.290436  0.041641  0.224758  1.000000  0.067348  0.211770  0.099292   
6        0.123769  0.157762  0.106698  0.067348  1.000000  0.134231  0.026378   
...           ...       ...       ...       ...       ...       ...       ...   
176101   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
177593   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
180095   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
180777   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
185135   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

movieId    10      11      12      16      18      19      22      23      \
movieId                                                                     
1        0.181953     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2        0.136826     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3        0.199611     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5        0.138500     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
6        0.446936     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...           ...     ...     ...     ...     ...     ...     ...     ...   
176101   0.000000     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
177593   0.000000     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
180095   0.000000     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
180777   0.000000     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
185135   0.000000     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  24      26      27      29      30      31      32      34      \
movieId                                                                   
1           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
6           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...         ...     ...     ...     ...     ...     ...     ...     ...   
176101      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
177593      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
180095      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
180777      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
185135      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  36      39      47      48      50      58      60      62      \
movieId                                                                   
1           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
6           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...         ...     ...     ...     ...     ...     ...     ...     ...   
176101      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
177593      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
180095      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
180777      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
185135      0.0     0.0     0.0     0.0     0.0     0.0     0.0     

In [30]:
def top_movies(movie):
    movies = []
    #if user not in user_sim_df.columns:
      #  return('No data available on user {}'.format(user))
    
    sim_values = movie_ratings.sort_values(by = movie, ascending=False).loc[:,movie].tolist()[1:20]
    sim_users = movie_ratings.sort_values(by = movie, ascending=False).index[1:20]
    zipped = zip(sim_users, sim_values,)
    for movie, sim in zipped:
        movies.append(movie)
    #movies = sim_movies(users, user_movie)
    return movies

In [31]:
watched_movies = data[data["userId"] == 1]

In [32]:
watched_movie_ids = watched_movies["movieId"]

In [33]:
sim_movies = []
for Id in watched_movie_ids:
    sim_movies.append(top_movies(Id))

In [34]:
sim_movies = list(itertools.chain.from_iterable(sim_movies))

In [35]:
for movie in sim_movies:
    print(df_movies[df_movies["movieId"] == movie]["title"].tolist())

['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['Toy Story (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, Th

['Tom and Huck (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Sabrina (1995)']
['GoldenEye (1995)']
['Toy Story (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Grumpier Old Men (1995)']
['Tom and Huck (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Sabrina (1995)']
['Toy Story (1995)']
['Ju

['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Toy Story (1995)']
['Grumpier Old Men (1995)']
['Sabrina (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American Presi

['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Toy Story (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (19

['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Toy Story (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)'

['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Jumanji (1995)']
['Grum

['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Cas

['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao)

['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Toy Story (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When N

['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
['Toy Story (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Dracula: Dead and Loving It (1995)']
['Casino (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Othello (1995)']
['Now and Then (1995)']
['City of Lost Children, The (Cité des enfants perdus, La) (1995)']
['Shanghai Triad (Yao a yao yao dao waipo qia

In [36]:
intersect_movies = list(set(watched_movies["movieId"]).intersection(sim_movies))

In [37]:
y_pred = []
y_test = []

for movie in intersect_movies:
    y_pred.append(df[movie])
    y_test.append(watched_movies[watched_movies["movieId"] == movie]["rating"].tolist())

In [38]:
y_pred

[3.857142857142857, 4.0, 4.0]

In [39]:
y_test

[[4.0], [4.0], [4.0]]

In [40]:
import math
import sklearn.metrics as metrics
print(np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_pred)))

0.08247860988423221


train_data = data["title"]
train_data = list(dict.fromkeys(train_data))

----------------------------------------------------------------------------------

In [41]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [42]:
spark = SparkSession.builder.appName('Recommendation_system').getOrCreate()

In [43]:
df = spark.read.csv("movie_recommendation.csv" , inferSchema = True , header = True)
df.show(100,truncate=True)

+------+-------+------+--------+----------------+--------------------+--------------------+--------------+
|userId|movieId|rating|    date|           title|              genres|                link|published year|
+------+-------+------+--------+----------------+--------------------+--------------------+--------------+
|     1|      1|   4.0|30-07-00|Toy Story (1995)|Adventure|Animati...|www.imdb.com/titl...|        (1995)|
|     5|      1|   4.0|08-11-96|Toy Story (1995)|Adventure|Animati...|www.imdb.com/titl...|        (1995)|
|     7|      1|   4.5|25-01-05|Toy Story (1995)|Adventure|Animati...|www.imdb.com/titl...|        (1995)|
|    15|      1|   2.5|13-11-17|Toy Story (1995)|Adventure|Animati...|www.imdb.com/titl...|        (1995)|
|    17|      1|   4.5|18-05-11|Toy Story (1995)|Adventure|Animati...|www.imdb.com/titl...|        (1995)|
|    18|      1|   3.5|11-02-16|Toy Story (1995)|Adventure|Animati...|www.imdb.com/titl...|        (1995)|
|    19|      1|   4.0|08-08-00|Toy S

In [44]:
df_rating = df.select(df['userId'],df['movieId'],df['rating'])
df_rating.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     5|      1|   4.0|
|     7|      1|   4.5|
|    15|      1|   2.5|
|    17|      1|   4.5|
|    18|      1|   3.5|
|    19|      1|   4.0|
|    21|      1|   3.5|
|    27|      1|   3.0|
|    31|      1|   5.0|
|    32|      1|   3.0|
|    33|      1|   3.0|
|    40|      1|   5.0|
|    43|      1|   5.0|
|    44|      1|   3.0|
|    45|      1|   4.0|
|    46|      1|   5.0|
|    50|      1|   3.0|
|    54|      1|   3.0|
|    57|      1|   5.0|
+------+-------+------+
only showing top 20 rows



In [45]:
(training, test) = df_rating.randomSplit([0.8, 0.2])

In [46]:
als_model = ALS(maxIter = 5,regParam = 0.005,rank = 25,userCol = "userId",itemCol = "movieId",ratingCol = "rating",coldStartStrategy = "drop")
model = als_model.fit(df_rating)

In [47]:
predictions = model.transform(test)

In [48]:
evaluator = RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("RMSE="+str(rmse))

RMSE=0.35002183828811195


In [49]:
user_recs = model.recommendForAllUsers(10)

In [50]:
spark.createDataFrame 
result_recomendation_user = user_recs.select("*").toPandas() 

In [51]:
result_recomendation_user["recommendations"][0]

[Row(movieId=2686, rating=8.177362442016602),
 Row(movieId=47999, rating=7.955020427703857),
 Row(movieId=3551, rating=7.723898887634277),
 Row(movieId=7004, rating=7.7075395584106445),
 Row(movieId=5419, rating=7.561461448669434),
 Row(movieId=3062, rating=7.528609275817871),
 Row(movieId=5784, rating=7.515812873840332),
 Row(movieId=3785, rating=7.48712158203125),
 Row(movieId=82459, rating=7.43032693862915),
 Row(movieId=134853, rating=7.375943660736084)]

In [52]:
movie_recs = model.recommendForAllItems(10)
spark.createDataFrame 
result_recomendation_movie = movie_recs.select("*").toPandas() 

In [53]:
result_recomendation_movie["recommendations"][0]

[Row(userId=77, rating=7.7244486808776855),
 Row(userId=472, rating=7.4104437828063965),
 Row(userId=584, rating=6.085110664367676),
 Row(userId=267, rating=5.8425798416137695),
 Row(userId=319, rating=5.7718281745910645),
 Row(userId=327, rating=5.7598114013671875),
 Row(userId=296, rating=5.748956680297852),
 Row(userId=491, rating=5.748330593109131),
 Row(userId=574, rating=5.668920516967773),
 Row(userId=56, rating=5.593020439147949)]